### Instructions for building field maps

1. Begin with a blank template like:

```python
# Oregon
state = fieldmaps.State('OR')

layout = {
    'api_field': '',
    'date_field': '',
    'lat_field': '',
    'lon_field': '',
    'status_field': '',
    'status_map': None,
    'type_field': '',
    'type_map': None
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, )
wells.write_to_db()
```

  * The API and Date fields can be strings or lists of strings (which will be evaluated in the order given).
  * If the date is in JavaScript epoch time, add `webdate = True` to the state object declaration
2. Fill in the fields (except status_map and type_map) from the provided list
3. Change the second to last line to: `wells = fieldmaps.Dataset(state, layout)`
4. Overwrite the status_map and type_map lines with the output, and fill in the appropriate values

A (likely incomplete) list of valid types:
well_types = {'OIL',
    'GAS',
    'OILANDGAS',
    'INJECTION',
    'STORAGE',
    'DISPOSAL',
    'SERVICE',
    'DRY HOLE',
    'OBSERVATION',
    'TEST',
    'WATER',
    'OTHER',
    'UNKNOWN'}
well_statuses = {'ACTIVE', 'INACTIVE', 'A', 'PA', 'TA', 'SI', 'DRY', 'ORPHAN', 'PERMITTED', 'CANCELLED', 'UNKNOWN'} # PA: plugged and abandoned, TA: temporarily abandoned
well_categories = {'CONVENTIONAL','UNCONVENTIONAL','FRAC','CBM','SERVICE','TEST','STORAGE','WATER','OTHER','UNKNOWN'}


In [1]:
# module that imports CSVs and processes to DB
import os, errno, array, csv, math, re
from datetime import datetime
import dateutil.parser
import importlib
import psycopg2
import fieldmaps

In [ ]:
# Alabama
import fieldmaps.al
state = fieldmaps.State('AL', webdate = True)
well_source_url = 'http://www.gsa.state.al.us/ogb/results.aspx?api=%s&permit=%s'
state.source_url = 'http://www.gsa.state.al.us/ogb/db_main.html'
state.description = """
Data from the Geological Survey of Alabama State Oil and Gas Board
"""
state.write_to_db()

dataset = fieldmaps.Dataset(state, fieldmaps.al.layout)
db_result = dataset.write_to_db()

In [ ]:
# Alaska
import fieldmaps.ak
state = fieldmaps.State('AK')
state.projection = 4267 # NAD27
state.source_url = 'http://aogweb.state.ak.us/DataMiner3/Forms/WellList.aspx'
state.description = """
Data from the Alaska Oil and Gas Conservation Commission
"""
state.write_to_db()

dataset = fieldmaps.Dataset(state, fieldmaps.ak.layout)
dataset.write_to_db()

In [ ]:
# Arkansas
import fieldmaps.ar
state = fieldmaps.State('AR')
state.source_url = 'http://www.noggateway.org/reports'
state.description = "Data from the National Oil and Gas Gateway"
state.write_to_db()

state.projection = 4269 # NAD83

dataset = fieldmaps.Dataset(state, fieldmaps.ar.layout)
dataset.write_to_db()

In [ ]:
# Arizona
import fieldmaps.az
state = fieldmaps.State('AZ')
state.source_url = 'http://services.azgs.az.gov/arcgis/rest/services/aasggeothermal/AZWellHeaders/MapServer/0'
state.description = """
Data from the Arizona Oil and Gas Conservation Commission ArcGIS map at http://ogviewer.azdeq.gov/.

Dates do not seem to be available from this source.
"""
state.date_function = lambda x: None
state.write_to_db()

dataset = fieldmaps.Dataset(state, fieldmaps.az.layout)
db_result = dataset.write_to_db()

In [ ]:
# California
import fieldmaps.ca
state = fieldmaps.State('CA', filename = 'california-data.csv', prepend_api = True, webdate = True)
state.source_url = 'http://spatialservices.conservation.ca.gov/arcgis/rest/services/DOMS/Wells/MapServer/0'
state.description = """California Division of Oil, Gas, & Geothermal Resources 
at http://www.conservation.ca.gov/dog. Type codes: ftp://ftp.consrv.ca.gov/pub/oil/GIS/ReadMe.pdf"""
state.write_to_db()

dataset = fieldmaps.Dataset(state, fieldmaps.ca.layout)
dataset.write_to_db()

In [ ]:
# Colorado
import fieldmaps.co
state = fieldmaps.State('CO')

wells = fieldmaps.Dataset(state, fieldmaps.co.layout)
wells.write_to_db()

In [ ]:
# Connecticut - No Oil or Gas Drilling

In [ ]:
# Delaware - No Oil or Gas Drilling

In [ ]:
# District of Columbia - No Oil or Gas Drilling

In [ ]:
# Florida
state = fieldmaps.State('FL', webdate = True,  prepend_api = True)
import fieldmaps.fl

wells = fieldmaps.Dataset(state, fieldmaps.fl.layout, quiet = True)
wells.write_to_db()

In [ ]:
# Georgia - No oil or gas activity

In [ ]:
# Hawaii - No oil or gas activity

In [ ]:
# Idaho
state = fieldmaps.State('ID')
# import fieldmaps.id
state.load_data()
layout = {
    'api_field': 'API',
    'date_field': 'Date',
    'lat_field': 'lattitudeWGS84',
    'lon_field': 'longitudeWGS84',
    'status_field': 'Status',
    'status_map': {
        'Not Drilled - Permit Expired': 'CANCELLED',
        'Permitted Well': 'PERMITTED',
        'Expired Permit': 'CANCELLED',
        'Plugged and Abandoned': 'PA',
        'Shut In': 'SI',
        'Not Drilled': 'PERMITTED',
        'Drilling': 'ACTIVE',
        'Producing': 'ACTIVE',
        'Completed': 'ACTIVE',
    },
    'type_field': 'Well Type',
    'type_map': {
        '': 'UNKNOWN',
        'Expired Location': 'UNKNOWN',
        'Oil Well': 'OIL',
        'Oil & Gas': 'OILANDGAS',
        'Gas Well': 'GAS',
        'Dry Hole': 'DRY HOLE',
    }
}
wells = fieldmaps.Dataset(state, layout, quiet = True)
wells.write_to_db()

In [ ]:
# Illinois
state = fieldmaps.State('IL', webdate = True, filename = 'il-data-old.csv')
import fieldmaps.il
wells = fieldmaps.Dataset(state, fieldmaps.il.layout, quiet = True)
wells.write_to_db()

In [ ]:
# Indiana
state = fieldmaps.State('IN', prepend_api = True)
import fieldmaps.indiana
wells = fieldmaps.Dataset(state, fieldmaps.indiana.layout)
wells.write_to_db()

In [ ]:
# Iowa - No oil or Gas

In [ ]:
# Kansas
state = fieldmaps.State('KS', webdate = True)
state.projection = 4269 # NAD83

import fieldmaps.ks
layout = fieldmaps.ks.layout
wells = fieldmaps.Dataset(state, layout, quiet = True)
wells.write_to_db()

In [ ]:
# Kentucky
state = fieldmaps.State('KY')
state.projection = 4267 # NAD27

import fieldmaps.ky
wells = fieldmaps.Dataset(state, fieldmaps.ky.layout)
wells.write_to_db()

In [ ]:
# Louisiana

state = fieldmaps.State('LA', webdate = True)
import fieldmaps.la

wells = fieldmaps.Dataset(state, fieldmaps.la.layout, quiet = True)
wells.write_to_db()

In [ ]:
# Maine - No Oil or Gas

In [ ]:
# Maryland - gas only
state = fieldmaps.State('MD', prepend_api = True)
layout = {
    'api_field': 'Id',
    'date_field': 'Date_drill',
    'lat_field': 'Y',
    'lon_field': 'X',
    'status_field': 'Status',
    'status_map': {
        '?': 'UNKNOWN',
        'Abandoned': 'A',
        'Active': 'ACTIVE',
        'Plugged': 'PA',
        'Plugged?': 'PA',
        'plugged': 'PA',
    },
    'type_field': 'Status',
    'type_map': {
        '?': 'GAS',
        'Abandoned': 'GAS',
        'Active': 'GAS',
        'Plugged': 'GAS',
        'Plugged?': 'GAS',
        'plugged': 'GAS',
    },
}
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Massachusetts - No oil or gas

In [ ]:
# Michigan
# note - these are bottom hole locations. Getting surface locations
# and joining dates would take some work
state = fieldmaps.State('MI', webdate = True)
layout = {
    'api_field': 'api_wellno',
    'date_field': ['PermDate', 'PlugDate'],
    'lat_field': 'bh_lat',
    'lon_field': 'bh_long',
    'status_field': 'well_stat',
    'status_map': {
        'Orphan': 'ORPHAN',
        'Pilot': 'UNKNOWN',
        'Plugged Back': 'PA',
        'Plugging Approved': 'PA',
        'Plugging Completed': 'PA',
        'Producing': 'ACTIVE',
        'Shut_In': 'SI',
        'Suspended Operations': 'INACTIVE',
        'Temporarily Abandoned': 'TA',
        'Terminated Permit': 'CANCELLED',
        'Unknown': 'UNKNOWN',
        'Well Completed': 'ACTIVE',
    },
    'type_field': 'well_type',
    'type_map': {
        'GAS': 'GAS',
        'GC': 'GAS',
        'OIL': 'OIL',
    },
}

wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Minnesota - no oil or gas

In [ ]:
# Mississippi
state = fieldmaps.State('MS')
layout = {
   'date_field': ['SPUD DATE', 'FIRST COMPLETION DATE', 'FIRST PRODUCTION DATE'],
    'api_field': 'API NUMBER',
    'type_field': 'STANDARD WELL TYPE DESCRIPTION',
    'status_field': 'STANDARD WELL STATUS DESCRIPTION',
    'lat_field': 'WELLHEAD LATITUDE, DECIMAL DEGREES',
    'lon_field': 'WELLHEAD LONGITUDE, DECIMAL DEGREES',
    'status_map': {
        'Unassigned': 'UNKNOWN',
        'Abandoned': 'A',
        'Abandoned:Plugged': 'PA',
        'Abandoned:Dry Hole': 'A',
        'Active:Producing': 'ACTIVE',
        'Permit:Active': 'ACTIVE',
        'Inactive:Temporarily Abandoned': 'TA',
        'Permit:Cancelled Expired': 'CANCELLED',
        'Active:Completed': 'ACTIVE',
        'Active': 'ACTIVE',
        'Inactive': 'A',
        'Active:Injecting': 'ACTIVE',
    },
    'type_map': {
        'Other': 'OTHER',
        'Gas': 'GAS',
        'Unassigned': 'OTHER',
        'Oil': 'OIL',
        'Observation Monitor Strat Test': 'TEST',
        'Gas Storage': 'STORAGE',
        'Water Supply': 'WATER',
        'Permit': 'UNKNOWN',
        'Underground Injection Control:Class 2 Disposal': 'INJECTION',
        'Dry Hole': 'DRY HOLE',
        'Underground Injection Control:Class 2 Enhanced Recovery': 'INJECTION',
        'Underground Injection Control:Class 1': 'INJECTION',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Missouri
state = fieldmaps.State('MO', strip_whitespace = ['Well Status', 'Well Type'], prepend_api = True)
#import fieldmaps.ks
layout = {
    'api_field': 'API Number',
    'date_field': ['Spud Date','Well Status Date'],
    'lat_field': 'Well Latitude Decimal',
    'lon_field': 'Well Longitude Decimal',
    'status_field': 'Well Status',
    'status_map': {
        'Abandoned': 'A',
        'Abandoned, Known Location and Verified': 'A',
        'Abandoned, No evidence of existence/ Unable to find': 'A',
        'Abandoned, Unknown Location': 'A',
        'Active Well': 'ACTIVE',
        'Inactive Ð Initial 90 day window': 'TA',
        'Plugged - Approved': 'PA',
        'Proposed Well (Associated with permit application only)': 'UNKNOWN',
        'Shut in - Complete': 'SI',
        'Shut in - Extended': 'SI',
        'Shut in - Incomplete': 'SI',
        'Temporarily Abandoned(Idle)': 'TA',
        'Under Construction': 'PERMITTED',
        'Unknown Well Status': 'UNKNOWN',
        'Water Well Conversion': 'UNKNOWN',
    },
    'type_field': 'Well Type',
    'type_map': {
        'Dry Hole': 'DRY HOLE',
        'Gas Storage Wells': 'STORAGE',
        'Gas(Coalbed Methane)': 'GAS',
        'Gas(Convertional, Commercial)': 'GAS',
        'Gas(Private Use)': 'GAS',
        'Horizontal Oil Well': 'OIL',
        'Injection(Class II Disposal)': 'INJECTION',
        'Injection(Class II EOR)': 'INJECTION',
        'Observation': 'OBSERVATION',
        'Oil': 'OIL',
        'Stratigraphic Test': 'TEST',
        'Stratigraphic Test, None Oil/Gas Related(has geophysical log)': 'TEST',
        'Unknown Well Type': 'UNKNOWN',
        'Water Well': 'WATER',
        'Water Well, None Oil/Gas Related(has geophysical log)': 'WATER',
    },
}
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Montana
state = fieldmaps.State('MT')
#import fieldmaps.ks
layout = {
   'date_field': 'Dt_Cmp',
    'api_field': 'API #',
    'type_field': 'Well_Type',
    'status_field': 'Wl_Status',
    'lat_field': 'Wh_Lat',
    'lon_field': 'Wh_Long',
    'status_map': {
        'Abandoned': 'A',
        'Abandoned - Unapproved': 'A',
        'Active Injection': 'ACTIVE',
        'Completed': 'ACTIVE',
        'Domestic': 'ACTIVE',
        'Expired Permit': 'CANCELLED',
        'Expired, Not Released': 'CANCELLED',
        'Not Completed': 'PERMITTED',
        'Other': 'UNKNOWN',
        'P&A - Approved': 'PA',
        'Permit to Drill': 'PERMITTED',
        'Permitted Injection Well': 'PERMITTED',
        'Producing': 'ACTIVE',
        'Revoked Inj. Permit': 'CANCELLED',
        'Shut In': 'SI',
        'Spudded': 'ACTIVE',
        'Temporarily Abandoned': 'TA',
        'Unknown': 'UNKNOWN',
        'Water Well, Completed': 'ACTIVE',
        'Water Well, Released': 'ACTIVE',
    },
    'type_map': {
        'Class V Injection': 'INJECTION',
        'Coal Bed Methane': 'GAS',
        'Domestic Water': 'WATER',
        'Dry Hole': 'DRY HOLE',
        'Expired Location': 'UNKNOWN',
        'Gas': 'GAS',
        'Gas Storage': 'STORAGE',
        'Injection - Disposal': 'DISPOSAL',
        'Injection, EOR': 'INJECTION',
        'Injection, Indian Lands': 'INJECTION',
        'Monitor/Observation': 'OBSERVATION',
        'Oil': 'OIL',
        'Oil & Gas': 'OILANDGAS',
        'Stratigraphic Test': 'TEST',
        'Unknown': 'UNKNOWN',
        'Water Source': 'WATER',
    },
}
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Nebraska
state = fieldmaps.State('NE')

layout = {
   'date_field': ['SPUD DATE', 'FIRST COMPLETION DATE', 'FIRST PRODUCTION DATE'],
    'api_field': 'API NUMBER',
    'type_field': 'STANDARD WELL TYPE DESCRIPTION',
    'status_field': 'STANDARD WELL STATUS DESCRIPTION',
    'lat_field': 'WELLHEAD LATITUDE, DECIMAL DEGREES',
    'lon_field': 'WELLHEAD LONGITUDE, DECIMAL DEGREES',
    'status_map': {
        'Abandoned': 'A',
        'Abandoned:Dry Hole': 'A',
        'Abandoned:Plugged': 'PA',
        'Active': 'ACTIVE',
        'Active:Injecting': 'ACTIVE',
        'Active:Producing': 'ACTIVE',
        'Inactive': 'A',
        'Inactive:Shut In': 'SI',
        'Inactive:Temporarily Abandoned': 'TA',
        'Permit:Active': 'ACTIVE',
        'Permit:Cancelled Expired': 'CANCELLED',
        'Unassigned': 'UNKNOWN',
        'Unknown': 'UNKNOWN',
    },
    'type_map': {
        'Abandoned': 'UNKNOWN',
        'Dry Hole': 'DRY HOLE',
        'Gas': 'GAS',
        'Gas Storage': 'STORAGE',
        'Observation Monitor Strat Test': 'TEST',
        'Oil': 'OIL',
        'Other': 'OTHER',
        'Permit': 'UNKNOWN',
        'Unassigned': 'UNKNOWN',
        'Underground Injection Control:Class 1': 'INJECTION',
        'Underground Injection Control:Class 2': 'INJECTION',
        'Underground Injection Control:Class 2 Disposal': 'DISPOSAL',
        'Underground Injection Control:Class 2 Enhanced Recovery': 'INJECTION',
        'Water Supply': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Nevada
state = fieldmaps.State('NV') # strip_whitespace = ['Well Status', 'Well Type']
state.projection = 26910
layout = {
    'api_field': 'API',
    'date_field': 'COMPL_DATE',
    'lat_field': 'APROX_UTMN',
    'lon_field': 'APROX_UTME',
    'status_field': 'STATUS',
    'status_map': {
        ' ': 'UNKNOWN',
        'A': 'A',
        'Abandoned': 'A',
        'Cancelled': 'CANCELLED',
        'D & A': 'A',
        'D & A (?)': 'A',
        'D & A ?': 'A',
        'D&A': 'A',
        'D&A(?)': 'A',
        'D&A,Water well': 'A',
        'Drilled': 'ACTIVE',
        'Drilled?': 'ACTIVE',
        'Expired': 'CANCELLED',
        'Geothermal': 'ACTIVE',
        'Geothermal Wel': 'ACTIVE',
        'Injection': 'ACTIVE',
        'J & A': 'A',
        'J & A ?': 'A',
        'J&A ?': 'A',
        'Lease closed': 'CANCELLED',
        'Never Drilled': 'CANCELLED',
        'Never drilled': 'CANCELLED',
        'Never drilled?': 'CANCELLED',
        'P & A': 'PA',
        'P & A (?)': 'PA',
        'P & A 7/17/95': 'PA',
        'P & A 7/27/95': 'PA',
        'P&A': 'PA',
        'P&A (Producer)': 'PA',
        'Permit Revoked': 'CANCELLED',
        'Prod?Inject': 'ACTIVE',
        'Producer': 'ACTIVE',
        'Producer(?)': 'ACTIVE',
        'Shut In': 'SI',
        'Shut in': 'SI',
        'Suspended': 'TA',
        'TA': 'TA',
        'TA ?': 'TA',
        'Temp. Shut In': 'TA',
        'Testing': 'ACTIVE',
        'Water Disposal': 'ACTIVE',
        'Water Well': 'ACTIVE',
        'Water well': 'ACTIVE',
        'drilled': 'ACTIVE',
        'injection': 'ACTIVE',
        'injection well': 'ACTIVE',
        'producer': 'ACTIVE',
        're-entry': 'ACTIVE',
        'water well': 'ACTIVE',
    },
    'type_field': 'SHOW',
    'type_map': {
        ' ': 'UNKNOWN',
        '?': 'UNKNOWN',
        'GAS': 'GAS',
        'GAS  OIL': 'OILANDGAS',
        'GAS  WATER': 'GAS',
        'GAS WATER': 'GAS',
        'OIL': 'OIL',
        'OIL   GAS': 'OILANDGAS',
        'OIL  GAS': 'OILANDGAS',
        'OIL  GAS  WATER': 'OILANDGAS',
        'OIL  WATER': 'OIL',
        'OIL  WATER  GAS': 'OILANDGAS',
        'OIL  `': 'OIL',
        'OIL GAS': 'OIL',
        'OIL GAS WATER': 'OILANDGAS',
        'OIL WATER': 'OIL',
        'OIL, WATER': 'OIL',
        'OIL?': 'OIL',
        'Oil': 'OIL',
        'WATER': 'WATER',
        'WATER  GAS': 'GAS',
        'WATER  HELIUM G': 'OTHER',
        'WATER  OIL': 'OIL',
        'WATER  OIL GAS': 'OILANDGAS',
        'Water': 'WATER',
        'weak oil': 'OIL',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# New Hampshire - no oil or gas

In [ ]:
# New Jersey - no oil or gas

In [ ]:
# New Mexico
# I think the data is actually in WGS84 b/c that's how I requested
# it from the NM ArcGIS server
# status map: ftp://164.64.106.6/Public/OCD/OCD%20GIS%20Data/Metadata/NM_Wells_Metadata.pdf
state = fieldmaps.State('NM')
layout = {
    'api_field': 'API',
    'date_field': 'spud_date',
    'lat_field': 'latitude',
    'lon_field': 'longitude',
    'status_field': 'status',
    'status_map': {
        '0': 'UNKNOWN',
        'A': 'ACTIVE',
        'C': 'CANCELLED',
        'D': 'A',
        'E': 'CANCELLED',
        'H': 'A',
        'N': 'PERMITTED',
        'P': 'PA',
        'Q': 'PA',
        'S': 'SI',
        'T': 'TA',
        'X': 'PERMITTED',
        'Z': 'TA',
    },
    'type_field': 'well_type',
    'type_map': {
        'C': 'OILANDGAS',
        'G': 'GAS',
        'I': 'INJECTION',
        'M': 'OTHER',
        'O': 'OIL',
        'S': 'DISPOSAL',
        'W': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# New York
state = fieldmaps.State('NY', webdate = True)
state.projection = 4269 # NAD83
# STATUS CODES LOOKUP https://www.dec.ny.gov/energy/30438.html
layout = {
    'api_field': 'API_WELLNO',
    'date_field': ['DATE_SPUDDED', 'DATE_PERMIT_APPLICATION', 'DATE_WELL_COMPLETED'],
    'lat_field': 'SURFACE_LATITUDE',
    'lon_field': 'SURFACE_LONGITUDE',
    'status_field': 'WELL_STATUS',
    'status_map': {
        'AC': 'ACTIVE',
        'AR': 'UNKNOWN',
        'CA': 'CANCELLED',
        'CO': 'ACTIVE',
        'Confidential': 'ACTIVE',
        'DC': 'ACTIVE',
        'DD': 'ACTIVE',
        'DG': 'ACTIVE',
        'EX': 'CANCELLED',
        'IN': 'INACTIVE',
        'NR': 'UNKNOWN',
        'PA': 'PA',
        'PB': 'PA',
        'PI': 'PERMITTED',
        'PM': 'PA',
        'RE': 'CANCELLED',
        'SI': 'SI',
        'TA': 'TA',
        'TR': 'UNKNOWN',
        'UL': 'UNKNOWN',
        'UM': 'UNKNOWN',
        'UN': 'UNKNOWN',
        'VP': 'CANCELLED',
    },
    'type_field': 'WELL_TYPE',
    'type_map': {
        'BR': 'WATER',
        'Confidential': 'UNKNOWN',
        'DH': 'DRY HOLE',
        'DS': 'DISPOSAL',
        'DW': 'DRY HOLE',
        'GD': 'GAS',
        'GE': 'GAS',
        'GW': 'GAS',
        'IW': 'INJECTION',
        'LP': 'STORAGE',
        'MB': 'OBSERVATION',
        'MM': 'OBSERVATION',
        'MS': 'OBSERVATION',
        'NL': 'OTHER',
        'OD': 'OIL',
        'OE': 'OIL',
        'OW': 'OIL',
        'SG': 'TEST',
        'ST': 'STORAGE',
        'TH': 'OTHER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# North Carolina - no oil or gas

In [ ]:
# North Dakota
state = fieldmaps.State('ND')
# code definitions: https://www.dmr.nd.gov/oilgas/codehelp.asp
layout = {
    'api_field': 'api',
    'date_field': 'spud_date',
    'lat_field': 'latitude',
    'lon_field': 'longitude',
    'status_field': 'status',
    'status_map': {
        'A': 'ACTIVE',
        'AB': 'A',
        'Confidential': 'ACTIVE',
        'DRL': 'ACTIVE',
        'DRY': 'DRY',
        'EXP': 'CANCELLED',
        'IA': 'INACTIVE',
        'IAW': 'INACTIVE',
        'LOC': 'PERMITTED',
        'LOCR': 'PERMITTED',
        'NC': 'PERMITTED',
        'NCW': 'PERMITTED',
        'PA': 'PA',
        'PNC': 'CANCELLED',
        'PNS': 'CANCELLED',
        'TA': 'TA',
        'TAO': 'TA',
        'TASC': 'TA',
        'TATD': 'TA',
    },
    'type_field': 'well_type',
    'type_map': {
        'AGD': 'DISPOSAL',
        'AI': 'INJECTION',
        'CBM': 'GAS',
        'Confidential': 'UNKNOWN',
        'DF': 'INJECTION',
        'DFP': 'INJECTION',
        'GASC': 'GAS',
        'GASD': 'GAS',
        'GASN': 'GAS',
        'GI': 'INJECTION',
        'GS': 'STORAGE',
        'INJP': 'INJECTION',
        'OG': 'OILANDGAS',
        'ST': 'TEST',
        'SWD': 'DISPOSAL',
        'WI': 'INJECTION',
        'WS': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Ohio
state = fieldmaps.State('OH')

layout = {
    'api_field': 'api',
    'date_field': ['commenced', 'completed', 'issued'],
    'lat_field': 'WH_LAT',
    'lon_field': 'WH_LONG',
    'status_field': 'WL_STATUS_DESC',
    'status_map': {
        'Active Injection': 'ACTIVE',
        'Cancelled': 'CANCELLED',
        'Domestic Well': 'ACTIVE',
        'Drilled, Inactive': 'INACTIVE',
        'Drilling': 'ACTIVE',
        'Dry and Abandoned': 'A',
        'Exempt Mississippian Well': 'UNKNOWN',
        'Field Inspected, Well Not Found': 'UNKNOWN',
        'Final Restoration': 'PA',
        'Historical Production Well': 'UNKNOWN',
        'Lost Hole': 'UNKNOWN',
        'Not Drilled': 'CANCELLED',
        'Orphan Well - Pending': 'ORPHAN',
        'Orphan Well - Ready': 'ORPHAN',
        'Plugged Back': 'PA',
        'Plugged and Abandoned': 'PA',
        'Producing': 'ACTIVE',
        'Replugged Well': 'PA',
        'Shut in': 'SI',
        'Storage Well': 'ACTIVE',
        'Temporary Inactive Well Status 1st Year': 'INACTIVE',
        'Temporary Inactive Well Status 2nd Year': 'INACTIVE',
        'Temporary Inactive Well Status 3rd Year': 'INACTIVE',
        'Unknown status': 'UNKNOWN',
        'Well Drilled': 'ACTIVE',
        'Well Permitted': 'PERMITTED',
    },
    'type_field': 'MapSymbol_DESC',
    'type_map': {
        'Brine for dust control': 'WATER',
        'Coalbed methane': 'GAS',
        'Dry hole': 'DRY HOLE',
        'Dry hole with gas show': 'GAS',
        'Dry hole with oil and gas show': 'GAS',
        'Dry hole with oil show': 'OIL',
        'Expired permit': 'UNKNOWN',
        'Gas': 'GAS',
        'Gas and oil show': 'OILANDGAS',
        'Gas show': 'GAS',
        'Gas storage': 'GAS',
        'Gas with oil show': 'OILANDGAS',
        'Injection': 'INJECTION',
        'Lost hole': 'UNKNOWN',
        'Observation': 'OBSERVATION',
        'Oil': 'OIL',
        'Oil and gas': 'OILANDGAS',
        'Oil and gas converted to water': 'WATER',
        'Oil show': 'OIL',
        'Oil with gas show': 'OILANDGAS',
        'Permitted location': 'UNKNOWN',
        'Plugged brine for dust control': 'WATER',
        'Plugged gas': 'GAS',
        'Plugged gas with oil show': 'OILANDGAS',
        'Plugged injection': 'INJECTION',
        'Plugged oil': 'OIL',
        'Plugged oil and gas': 'OILANDGAS',
        'Plugged oil with gas show': 'OILANDGAS',
        'Plugged water supply': 'WATER',
        'Plugged with gas show': 'GAS',
        'Plugged with oil show': 'OIL',
        'Radioactive tools lost in hole': 'UNKNOWN',
        'Solution mining': 'UNKNOWN',
        'Stratigraphy test': 'TEST',
        'Unknown status': 'UNKNOWN',
        'Water supply': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Oklahoma
# about half of Oklahoma's older wells are in TRS coordinates. I know
# once upon a time I had a function to convert from TRS to decimal degrees
# for the time being, these are getting ignored.
# also, unclear if this dataset includes the reservation or not.
state = fieldmaps.State('OK')

layout = {
   'date_field': ['SPUD DATE', 'FIRST COMPLETION DATE', 'FIRST PRODUCTION DATE'],
    'api_field': 'API NUMBER',
    'type_field': 'STANDARD WELL TYPE DESCRIPTION',
    'status_field': 'STANDARD WELL STATUS DESCRIPTION',
    'lat_field': 'WELLHEAD LATITUDE, DECIMAL DEGREES',
    'lon_field': 'WELLHEAD LONGITUDE, DECIMAL DEGREES',
    'status_map': {
        'Abandoned:Plugged': 'PA',
        'Active': 'ACTIVE',
        'Active:Completed': 'ACTIVE',
        'Inactive:Temporarily Abandoned': 'TA',
        'Permit:Active': 'ACTIVE',
        'Permit:Cancelled Expired': 'CANCELLED',
        'Unassigned': 'UNKNOWN',
        'Unknown': 'UNKNOWN',
    },
    'type_map': {
        'Abandoned': 'UNKNOWN',
        'Dry Hole': 'DRY HOLE',
        'Gas': 'GAS',
        'Gas Storage': 'STORAGE',
        'Oil': 'OIL',
        'Oil and Gas': 'OILANDGAS',
        'Other': 'OTHER',
        'Unassigned': 'UNKNOWN',
        'Underground Injection Control:Class 2': 'INJECTION',
        'Underground Injection Control:Class 2 Disposal': 'DISPOSAL',
        'Underground Injection Control:Class 2 Enhanced Recovery': 'INJECTION',
        'Unknown': 'UNKNOWN',
        'Water Supply': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Oregon
state = fieldmaps.State('OR')

layout = {
    'api_field': 'PermitID',
    'date_field': 'ApplicationDate',
    'lat_field': 'Latitude',
    'lon_field': 'Longitude',
    'status_field': 'Status',
    'status_map': {
        'Application': 'UNKNOWN',
        'Application Withdrawn': 'CANCELLED',
        'Cancelled': 'CANCELLED',
        'Closed': 'A',
        'Denied': 'CANCELLED',
        'Extended to 11/24/83': 'UNKNOWN',
        'Extended to 5/28/93': 'UNKNOWN',
        'Extended to 7/25/87': 'UNKNOWN',
        'Permitted': 'PERMITTED',
        'Terminated': 'A',
        'Unknown': 'UNKNOWN',
        'Withdrawn': 'CANCELLED',
    },
    'type_field': 'WellType',
    'type_map': {
        'Dry Hole': 'DRY HOLE',
        'Exploration': 'TEST',
        'Former producer - plugged': 'OIL',
        'Gas Injection/Withdrawal': 'INJECTION',
        'Gas Injection/Withdrawl': 'INJECTION',
        'Information': 'OBSERVATION',
        'Monitoring': 'OBSERVATION',
        'Never produced - plugged ': 'OIL',
        'Not Drilled': 'UNKNOWN',
        'Not drilled': 'UNKNOWN',
        'Observation': 'OBSERVATION',
        'Observation well': 'OBSERVATION',
        'Producer': 'OIL',
        'Water Disposal': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Pennsylvania
state = fieldmaps.State('PA')
state.source_url = 'http://www.depreportingservices.state.pa.us/ReportServer/Pages/ReportViewer.aspx?/Oil_Gas/Spud_External_Data'
state.description = """
Data downloaded in CSV form based on setting SPUD BEGIN DATE = 1/1/1800 and SPUD END DATE = 12/31/2099
Represents 142,788, of which 110,000 have dates wells.
"""
state.write_to_db()

layout = {
    'date_field': 'SPUD_DATE',
    'api_field': 'API',
    'type_field': 'WELL_CODE_DESC',
    'status_field': 'WELL_STATUS',
    'category_field': 'UNCONVENTIONAL',
    'lat_field': 'LATITUDE',
    'lon_field': 'LONGITUDE',
    'description_layout': '\n'.join(['County: %s',
                             'Operator Name: %s',
                             'OGO No: %s',
                             'Municipality: %s',
                             'Farm Name: %s',
                             'Well Configuration: %s'
                            ]),
    'description_fields': ['COUNTY', 'OPERATOR', 'OGO_NUM', 'MUNICIPALITY', 'FARM_NAME', 'CONFIGURATION'],
    'type_map': {
        'UNDETERMINED': 'OTHER',
        'MULTIPLE WELL BORE TYPE': 'OTHER',
        'OIL': 'OIL',
        'OBSERVATION': 'OBSERVATION',
        'DRY HOLE': 'DRY HOLE',
        'COALBED METHANE': 'GAS',
        'GAS': 'GAS',
        'COMB. OIL&GAS': 'OILANDGAS',
        'STORAGE WELL': 'STORAGE',
        'WASTE DISPOSAL': 'DISPOSAL',
        'INJECTION': 'INJECTION',
        'TEST WELL': 'TEST',
    },

    'status_map': {
        'DEP Abandoned List': 'ORPHAN',
        'Plugged OG Well': 'PA',
        'DEP Orphan List': 'ORPHAN',
        'DEP Plugged': 'PA',
        'Regulatory Inactive Status': 'UNKNOWN',
        'Operator Reported Not Drilled': 'CANCELLED',
        'Proposed But Never Materialized': 'CANCELLED',
        'Active': 'ACTIVE',
        'Abandoned': 'PA',
    },

    'category_map': {
        'Yes': 'UNCONVENTIONAL',
        'No': 'CONVENTIONAL',
    },

}

wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Rhode Island - no oil or gas

In [ ]:
# South Carolina - No oil or gas

In [ ]:
# South Dakota
state = fieldmaps.State('SD')
state.projection = 4269 # NAD83
layout = {
    'api_field': 'API Number',
    'date_field': ['Spud Date', 'Date Permit Issued'],
    'lat_field': 'Latitude (GCS83)',
    'lon_field': 'Longitude (GCS83)',
    'status_field': 'Administrative Status',
    'status_map': {
        'Abandoned-Not Regulated': 'A',
        'Converted-Domestic Gas': 'ACTIVE',
        'Converted-Fresh Water Supply': 'ACTIVE',
        'Converted-Water Supply-Injection': 'ACTIVE',
        'Injecting': 'ACTIVE',
        'Never drilled': 'CANCELLED',
        'Not Spudded': 'PERMITTED',
        'Plugged and Abandoned': 'PA',
        'Producing': 'ACTIVE',
        'Shut In': 'SI',
        'Temporarily Abandoned': 'TA',
    },
    'type_field': 'Permitted Well Type',
    'type_map': {
        'Gas': 'GAS',
        'Injection': 'INJECTION',
        'Injection-Horizontal': 'INJECTION',
        'Oil': 'OIL',
        'Oil and Gas Exploration': 'TEST',
        'Oil-Horizontal': 'OIL',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Tennessee
state = fieldmaps.State('TN')

state.description = """
Tennessee's Department of Environment & Conservation does not 
doesn't make a lot of details available (no spud date, well status). The dates recorded are the 
Permit Date. Status is set to unknown for all wells. Per their website, 
<i>"This minimal permit information is intended to allow the public to easily track the number and 
locations of oil and gas well permits issued. For more detailed information about a specific well, 
contact the Oil and Gas program staff at 615-687-7109 . Permit files can be accessed by visiting 
the Oil and Gas Section at the Nashville Environmental Field Office in Nashville, TN 37243."</i>
"""
state.source_url = 'http://environment-online.state.tn.us:8080/pls/enf_reports/f?p=9034:34300:0::NO:::'
state.write_to_db()

layout = {
    'date_field': 'Permit Date',
    'api_field': 'API No',
    'type_field': 'Purpose af Well',
    'status_field': 'Purpose af Well',
    'lat_field': 'Latitude',
    'lon_field': 'Longitude',
    'description_layout': '\n'.join(['County: %s',
                             'Operator Name: %s',
                             'Permit No: %s',
                             'Well Name and Number: %s',
                             'EFO Name: %s'
                            ]),
    'description_fields': ['County', 'Operator Name', 'Permit No', 'Well Name and Number', 'EFO Name'],
    'type_map': {
        'Gas': 'GAS',
        'Mineral Core Test': 'TEST',
        'NCG - Domestic Use': 'GAS',
        'Oil': 'OIL',
        'Oil And Gas': 'OILANDGAS',
    },
    'status_map': {
        'Gas': 'ACTIVE',
        'Mineral Core Test': 'UNKNOWN',
        'NCG - Domestic Use': 'ACTIVE',
        'Oil': 'ACTIVE',
        'Oil And Gas': 'ACTIVE',
    },
}

wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [2]:
# Texas
# ug - scraped permits, only came away with 1m of 2m spud dates
# and now for some reason I don't understand only half of Texas's wells are loading.
state = fieldmaps.State('TX', prepend_api = True)
state.projection = 4269 # NAD83
layout = {
    'api_field': 'API',
    'date_field': ['Approved Date', 'Submitted Date'],
    'lat_field': 'GIS_LAT83',
    'lon_field': 'GIS_LONG83',
    'status_field': 'GIS_SYMBOL_DESCRIPTION',
    'status_map': {
        'Brine Mining': 'ACTIVE',
        'Brine Mining / Oil': 'ACTIVE',
        'Brine Mining / Oil / Gas': 'ACTIVE',
        'Canceled / Abandoned Location': 'CANCELLED',
        'Core Test': 'ACTIVE',
        'Dry Hole': 'DRY',
        'Gas Well': 'ACTIVE',
        'Geothermal Well': 'ACTIVE',
        'Injection / Disposal': 'ACTIVE',
        'Injection / Disposal from Gas': 'ACTIVE',
        'Injection / Disposal from Oil': 'ACTIVE',
        'Injection / Disposal from Oil/Gas': 'ACTIVE',
        'Injection/Disposal from Brine Mining': 'ACTIVE',
        'Injection/Disposal from Brine Mining / Gas': 'ACTIVE',
        'Injection/Disposal from Storage': 'ACTIVE',
        'Injection/Disposal from Storage/Brine Mining': 'ACTIVE',
        'Injection/Disposal from Storage/Gas': 'ACTIVE',
        'Injection/Disposal from Storage/Oil': 'ACTIVE',
        'Observation Well': 'ACTIVE',
        'Observation from Gas': 'ACTIVE',
        'Observation from Oil': 'ACTIVE',
        'Observation from Oil / Gas': 'ACTIVE',
        'Observation from Storage': 'ACTIVE',
        'Observation from Storage/Gas': 'ACTIVE',
        'Observation from Storage/Oil': 'ACTIVE',
        'Oil Well': 'ACTIVE',
        'Oil/Gas Well': 'ACTIVE',
        'Permitted Location': 'PERMITTED',
        'Plugged Brine Mining': 'PA',
        'Plugged Gas Well': 'PA',
        'Plugged Oil / Gas': 'PA',
        'Plugged Oil Well': 'PA',
        'Plugged Storage': 'PA',
        'Plugged Storage/Brine Mining': 'PA',
        'Plugged Storage/Gas': 'PA',
        'Plugged Storage/Oil': 'PA',
        'Service': 'ACTIVE',
        'Service from Brine Mining': 'ACTIVE',
        'Service from Oil': 'ACTIVE',
        'Service from Oil / Gas': 'ACTIVE',
        'Service from Storage/Gas': 'ACTIVE',
        'Shut-In Gas': 'SI',
        'Shut-In Oil': 'SI',
        'Storage Well': 'ACTIVE',
        'Storage from Gas': 'ACTIVE',
        'Storage from Oil': 'ACTIVE',
        'Storage from Oil/Gas': 'ACTIVE',
        'Storage/Brine Mining': 'ACTIVE',
        'Sulfur Core Test': 'ACTIVE',
        'Water Supply Well': 'ACTIVE',
        'Water Supply from Gas': 'ACTIVE',
        'Water Supply from Oil': 'ACTIVE',
        'Water Supply from Oil/Gas': 'ACTIVE',
    },
    'type_field': 'GIS_SYMBOL_DESCRIPTION',
    'type_map': {
        'Brine Mining': 'OTHER',
        'Brine Mining / Oil': 'OIL',
        'Brine Mining / Oil / Gas': 'OILANDGAS',
        'Canceled / Abandoned Location': 'UNKNOWN',
        'Core Test': 'TEST',
        'Dry Hole': 'DRY HOLE',
        'Gas Well': 'GAS',
        'Geothermal Well': 'OTHER',
        'Injection / Disposal': 'DISPOSAL',
        'Injection / Disposal from Gas': 'DISPOSAL',
        'Injection / Disposal from Oil': 'DISPOSAL',
        'Injection / Disposal from Oil/Gas': 'DISPOSAL',
        'Injection/Disposal from Brine Mining': 'DISPOSAL',
        'Injection/Disposal from Brine Mining / Gas': 'DISPOSAL',
        'Injection/Disposal from Storage': 'DISPOSAL',
        'Injection/Disposal from Storage/Brine Mining': 'DISPOSAL',
        'Injection/Disposal from Storage/Gas': 'DISPOSAL',
        'Injection/Disposal from Storage/Oil': 'DISPOSAL',
        'Observation Well': 'OBSERVATION',
        'Observation from Gas': 'OBSERVATION',
        'Observation from Oil': 'OBSERVATION',
        'Observation from Oil / Gas': 'OBSERVATION',
        'Observation from Storage': 'OBSERVATION',
        'Observation from Storage/Gas': 'OBSERVATION',
        'Observation from Storage/Oil': 'OBSERVATION',
        'Oil Well': 'OIL',
        'Oil/Gas Well': 'OILANDGAS',
        'Permitted Location': 'UNKNOWN',
        'Plugged Brine Mining': 'UNKNOWN',
        'Plugged Gas Well': 'GAS',
        'Plugged Oil / Gas': 'OILANDGAS',
        'Plugged Oil Well': 'OIL',
        'Plugged Storage': 'STORAGE',
        'Plugged Storage/Brine Mining': 'STORAGE',
        'Plugged Storage/Gas': 'GAS',
        'Plugged Storage/Oil': 'OIL',
        'Service': 'SERVICE',
        'Service from Brine Mining': 'SERVICE',
        'Service from Oil': 'SERVICE',
        'Service from Oil / Gas': 'SERVICE',
        'Service from Storage/Gas': 'SERVICE',
        'Shut-In Gas': 'GAS',
        'Shut-In Oil': 'OIL',
        'Storage Well': 'STORAGE',
        'Storage from Gas': 'STORAGE',
        'Storage from Oil': 'STORAGE',
        'Storage from Oil/Gas': 'STORAGE',
        'Storage/Brine Mining': 'STORAGE',
        'Sulfur Core Test': 'TEST',
        'Water Supply Well': 'WATER',
        'Water Supply from Gas': 'WATER',
        'Water Supply from Oil': 'WATER',
        'Water Supply from Oil/Gas': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

1462802 of 1517493 rows loaded
first item: [{'state': 'TX', 'api': '42421114', 'lon': -102.12224025, 'lat': 36.45300986, 'date': '', 'type': 'UNKNOWN', 'status': 'PERMITTED', 'category': None, 'description': None}]
loaded 1462802 of 1462802 records into the database


1462802

In [ ]:
importlib.reload(fieldmaps)

layout = {
    'api_field': '',
    'date_field': '',
    'lat_field': '',
    'lon_field': '',
    'status_field': '',
    'status_map': None,
    'type_field': '',
    'type_map': None
}

In [ ]:
# Utah
state = fieldmaps.State('UT')
# type and status codes: https://gis.utah.gov/data/energy/oil-gas/
layout = {
   'date_field': 'OrigComplD',
    'api_field': 'API',
    'type_field': 'WellType',
    'status_field': 'WellStatus',
    'lat_field': 'Latitude',
    'lon_field': 'Longitude',
    'status_map': {
        'A': 'ACTIVE',
        'APD': 'PERMITTED',
        'DRL': 'ACTIVE',
        'I': 'INACTIVE',
        'LA': 'A',
        'NEW': 'UNKNOWN',
        'OPS': 'UNKNOWN',
        'P': 'ACTIVE',
        'PA': 'PA',
        'RET': 'CANCELLED',
        'S': 'SI',
        'SAI': 'SI',
        'TA': 'TA',
    },
    'type_map': {
        'CD': 'UNKNOWN',
        'D': 'DRY HOLE',
        'GI': 'INJECTION',
        'GS': 'STORAGE',
        'GW': 'GAS',
        'HE': 'OTHER',
        'NA': 'UNKNOWN',
        'OW': 'OIL',
        'TW': 'TEST',
        'WD': 'DISPOSAL',
        'WI': 'INJECTION',
        'WS': 'WATER',
        'tw': 'TEST',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Vermont - no oil or gas

In [ ]:
# Virginia
state = fieldmaps.State('VA', prepend_api = True, webdate = True)

layout = {
    'api_field': 'FiNo',
    'date_field': ['FiWellComp', 'FiStatDt'],
    'lat_field': 'Y',
    'lon_field': 'X',
    'status_field': 'TblPsDesc',
    'status_map': {
        'Completed': 'ACTIVE',
        'Drilled/Waiting Completion/PL': 'ACTIVE',
        'Drilling': 'ACTIVE',
        'Not Connected': 'UNKNOWN',
        'Orphaned Well': 'ORPHAN',
        'Other': 'UNKNOWN',
        'Partial Plug': 'A',
        'Plugged/Abandoned': 'PA',
        'Plugging': 'PA',
        'Producing': 'ACTIVE',
        'Released': 'UNKNOWN',
        'STABILIZED/OTHER': 'UNKNOWN',
        'STABILIZED/PRODUCING': 'ACTIVE',
        'Shut In': 'SI',
    },
    'type_field': 'TblOpDesc',
    'type_map': {
        'Coal Bed': 'GAS',
        'Coalbed/Pipeline': 'UNKNOWN',
        'Conversion': 'UNKNOWN',
        'Conversion to Water Well': 'WATER',
        'Conversion/Pipeline': 'UNKNOWN',
        'Exploratory': 'TEST',
        'Gas': 'GAS',
        'Gas/CB Dual Completion': 'GAS',
        'Gas/Coalbed with Pipeline': 'GAS',
        'Gas/Pipeline': 'GAS',
        'Geophysical': 'UNKNOWN',
        'Horizontal Coalbed': 'GAS',
        'Horizontal Coalbed W/PL': 'GAS',
        'Horizontal Gas': 'GAS',
        'Horizontal Gas W/PL': 'GAS',
        'Oil': 'OIL',
        'Service Well': 'SERVICE',
        'Underground Storage': 'STORAGE',
        'Waste Disposal': 'DISPOSAL',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Washington
state = fieldmaps.State('WA', webdate = True)

layout = {
    'api_field': 'API_NUMBER',
    'date_field': ['DRILL_START_DATE', 'PERMIT_DATE'],
    'lat_field': 'LATITUDE_DECIMAL_DEGREES',
    'lon_field': 'LONGITUDE_DECIMAL_DEGREES',
    'status_field': 'WELL_STATUS',
    'status_map': {
        'Completed': 'ACTIVE',
        'Completed, plugged, and abandoned': 'PA',
        'Converted to water well': 'ACTIVE',
        'Drilled': 'ACTIVE',
        'Plugged and abandoned': 'PA',
    },
    'type_field': 'HYDROCARBON_SHOW',
    'type_map': {
        'gas': 'GAS',
        'gas, oil': 'OILANDGAS',
        'oil': 'OIL',
    },

}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# West Virginia
state = fieldmaps.State('WV', webdate = True)

layout = {
    'api_field': 'api',
    'date_field': ['CompDate','IssueDate'],
    'lat_field': 'y',
    'lon_field': 'x',
    'status_field': 'WellStatus',
    'status_map': {
        'Abandoned Well': 'A',
        'Active Well': 'ACTIVE',
        'Drilling': 'ACTIVE',
        'Future Use': 'UNKNOWN',
        'Never Drilled': 'CANCELLED',
        'Never Issued': 'CANCELLED',
        'Permit Application': 'UNKNOWN',
        'Permit Issued': 'PERMITTED',
        'Plugged': 'PA',
        'Shutin': 'SI',
    },
    'type_field': 'WellUse',
    'type_map': {
        'Brine Disposal': 'DISPOSAL',
        'CBM - Articulate': 'GAS',
        'EOR Production Well': 'UNKNOWN',
        'Fluid Injection': 'INJECTION',
        'Gas Production': 'GAS',
        'House Gas ': 'GAS',
        'Not Available': 'UNKNOWN',
        'Observation': 'OBSERVATION',
        'Oil Production': 'OIL',
        'Production': 'OIL', # NOT SURE ABOUT THIS ONE
        'Solution Mining': 'UNKNOWN',
        'Storage': 'STORAGE',
        'Unknown': 'UNKNOWN',
        'Vent': 'UNKNOWN',
    },
}
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Wisconsin - no oil or gas

In [ ]:
# Wyoming
state = fieldmaps.State('WY')
# codes: http://wsgs.maps.arcgis.com/apps/webappviewer/index.html?id=3f7ab99343c34bd3ac5ae6ac8c04d95a
layout = {
    'api_field': 'APINO',
    'date_field': ['FIRSTSPUD', 'FIRSTCOMP'],
    'lat_field': 'LAT',
    'lon_field': 'LON',
    'status_field': 'STATUS',
    'status_map': {
        'AI': 'ACTIVE',
        'AP': 'PERMITTED',
        'D': 'ACTIVE',
        'DH': 'DRY',
        'DP': 'ACTIVE',
        'DR': 'ACTIVE',
        'FL': 'ACTIVE',
        'GL': 'ACTIVE',
        'GS': 'ACTIVE',
        'M': 'ACTIVE',
        'MW': 'ACTIVE',
        'ND': 'CANCELLED',
        'NI': 'A',
        'NR': 'UNKNOWN',
        'PA': 'PA',
        'PG': 'ORPHAN',
        'PH': 'ACTIVE',
        'PL': 'ACTIVE',
        'PO': 'ACTIVE',
        'PR': 'ACTIVE',
        'PS': 'ACTIVE',
        'SI': 'SI',
        'SO': 'SI',
        'SP': 'SI',
        'SR': 'SI',
        'TA': 'TA',
        'WP': 'UNKNOWN',
        'si': 'SI',
    },
    'type_field': 'WELL_CLASS',
    'type_map': {
        '01': 'DISPOSAL',
        '05': 'DISPOSAL',
        '1': 'DISPOSAL',
        'CB': 'GAS',
        'D': 'DISPOSAL',
        'DO': 'DISPOSAL',
        'G': 'GAS',
        'GO': 'GAS',
        'GS': 'STORAGE',
        'I': 'INJECTION',
        'IO': 'INJECTION',
        'LW': 'WATER',
        'M': 'OBSERVATION',
        'MO': 'OBSERVATION',
        'MW': 'OBSERVATION',
        'NA': 'UNKNOWN',
        'O': 'OIL',
        'OO': 'OIL',
        'S': 'WATER',
        'ST': 'TEST',
        'WS': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Alaska Offshore
state = State('AK1')
# Pacific Coast Offshore
state = State('CA1')
# Northern Gulf of Mexico
state = State('TX1')
# Atlantic Coast Offshore
state = State('DC1')
